In [9]:
import numpy as np
import tensorflow as tf

In [10]:
x = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
y = np.array([-3.0,-1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.compile(
    optimizer='sgd',
    loss='mean_squared_error'
)
model.fit(x, y, epochs=500)

Epoch 1/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step - loss: 6.7770
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 5.5205
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 4.5281
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 3.7436
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - loss: 3.1226
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 2.6304
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - loss: 2.2396
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 1.9287
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 1.6806
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 1.4821
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 1.3226
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - loss: 1.1939
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step - loss: 1.0895
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - loss: 1.0043
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.9342
Epoch 16/5

In [13]:
print(model.predict(np.array([10.0])))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[[18.98372]]


## Walkthrough: Creating and Converting a Model to TensorFlow Lite

## Step 1. Save the model

In [15]:
export_dir = '/content/drive/MyDrive/TfLite/saved_model/1'
tf.saved_model.save(model, export_dir)

## step 2. Convert and save model

In [25]:
# Create a concrete function (frozen model, no tf.Variable)
run_model = tf.function(lambda x: model(x))
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([None, 1], tf.float32))

# Convert using the concrete function
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = pathlib.Path("model.tflite")
tflite_model_path.write_bytes(tflite_model)

956

## step 3. Load the TFLite model and allocate Tensors

In [26]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

/usr/local/lib/python3.11/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [27]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'x', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'Identity', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [28]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)

[[10.]]


## step 4. Perform the prediction

In [29]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.98372]]
